In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import scipy
import gc
import numba



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.

#Initialize Variables
np.random.seed(1)
alpha = .05
omega = 0
smtooh_thresh = .5

network_num = 3
h_layer_num = 6
h_layere_num = 2

n_x = 800000
n_h = 400
n_y = 100
layers_dims = [n_x, n_y, n_h, h_layer_num]

ne_x = 300
ne_h = 100
ne_y = 1
layere_dims = [ne_x, ne_y, ne_h, h_layere_num]


num_batches = 12
batchsize_max = 200

[]
['input', 'lib', 'working', 'src']


In [2]:
#TODO 1.1
#Create full output for submissions
#Add convienence functions, plotting, type changes, etc.
#Add competition result evaluation function
#Set logic into classes?


#TODO 1.2
#initialize different random seed each time, use mod of time?
#change layer_dims to allow variable layers per unit
#change to new structure
#Add inhibitory functions
#Change to RELU activation
#allow for activations function change?
#check to make sure Y1[x] = Y2[x]

#TODO 1.3
#Harp style connections, angled out for access to outputs

#TODO for LTSM 
#change batch numbers to random number between 0 and total batches


In [3]:
#Format Input Training Data
def retrieve_traindata():
    rawmeta_train = pd.read_csv('../input/metadata_train.csv', dtype='int8')
    ph1_list = rawmeta_train.loc[rawmeta_train['phase'] == 0].copy()
    ph2_list = rawmeta_train.loc[rawmeta_train['phase'] == 1].copy()
    ph3_list = rawmeta_train.loc[rawmeta_train['phase'] == 2].copy()
    
    rawdata_train = pq.read_pandas('../input/train.parquet').to_pandas()
    ph1data_train = rawdata_train.iloc[:,ph1_list['signal_id'].values].copy()
    ph2data_train = rawdata_train.iloc[:,ph2_list['signal_id'].values].copy()
    ph3data_train = rawdata_train.iloc[:,ph3_list['signal_id'].values].copy()
    del rawdata_train
    Y1 = ph1_list['target'].copy()
    Y2 = ph2_list['target'].copy()
    Y3 = ph3_list['target'].copy()
    
    del ph1_list, ph2_list, ph3_list, rawmeta_train
    gc.collect()

    return ph1data_train, ph2data_train, ph3data_train, Y1, Y2, Y3



#Separate Training Data into Batches
def segment_traindata(num_batches, batchsize_max):
    try:
        os.mkdir('../train_batches')
    except FileNotFoundError:
        #file not found
        print("File not found, exiting...")
    else:
        ph1_data, ph2_data, ph3_data, train_Y1, train_Y2, train_Y3 = retrieve_traindata()
        
        n_x, m = ph1_data.shape
        mlist = list(range(0, m))
        
        ph1_data.columns = mlist
        train_Y1.index = mlist
        
        ph2_data.columns = mlist
        train_Y2.index = mlist
        
        ph3_data.columns = mlist
        train_Y3.index = mlist
        
        for i in range(num_batches):
            batch_size = np.random.randint(batchsize_max)
            batch_start = m
            
            while batch_start >= (m - batch_size):
                batch_start = np.random.randint(m)
            batch_end = batch_start+batch_size
            
            
            batch1 = ph1_data.iloc[:,batch_start:batch_end].copy()
            batch2 = ph2_data.iloc[:,batch_start:batch_end].copy()
            batch3 = ph3_data.iloc[:,batch_start:batch_end].copy()
        
            Y1 = train_Y1.iloc[batch_start:batch_end].copy()
            Y2 = train_Y2.iloc[batch_start:batch_end].copy()
            Y3 = train_Y3.iloc[batch_start:batch_end].copy()
        
            batch1.to_csv('../train_batches/ph1_' + str(i)+".csv")
            batch2.to_csv('../train_batches/ph2_' + str(i)+".csv")
            batch3.to_csv('../train_batches/ph3_' + str(i)+".csv")
        
            Y1.to_csv('../train_batches/y1_' + str(i)+".csv", header = None)
            Y2.to_csv('../train_batches/y2_' + str(i)+".csv", header = None)
            Y3.to_csv('../train_batches/y3_' + str(i)+".csv", header = None)
            
            
            print("Batch "+str(i+1)+" of "+str(num_batches))
            #print("batch1: ", batch1[0:10])
            #print("Y1: ", Y1[0:10])
            #print("Batch size: ", batch_size)
            #print("Batch start: ", batch_start)
            #print("Batch end: ", batch_end)
            #print("batch1: ", batch1.shape)
            #print("Y1: ", Y1.shape)
            #del batch1, batch2, batch3, Y1, Y2, Y3
            #gc.collect()
    
    del ph1_data, ph2_data, ph3_data, train_Y1, train_Y2, train_Y3
    gc.collect()
    return


#Convert CSV File Data into Numpy Array
def filetondarray(filename_data, filename_targets):
    data = pd.read_csv(filename_data, dtype='int8')
    Y = pd.read_csv(filename_targets, dtype='int8', header=None)
    #print(Y.head)
    #print(data.shape)
    #print(Y.shape)
    data_out = data.iloc[:,1:].values
    Y_out = Y.iloc[:,1:].values
    
    
    return data_out.copy(), Y_out.T.copy()

#Cleanup File Directory of old Batched Data
def file_cleanup():
    folder = '../train_batches/'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)
        
    #print(os.listdir("../train_batches"))
    
    os.rmdir('../train_batches')
    return

#Format Input Testing Data
def retrieve_testdata():
    rawmeta_test = pd.read_csv('../input/metadata_test.csv', dtype='int8')
    ph1_list = rawmeta_test.loc[rawmeta_test['phase'] == 0].copy()
    ph2_list = rawmeta_test.loc[rawmeta_test['phase'] == 1].copy()
    ph3_list = rawmeta_test.loc[rawmeta_test['phase'] == 2].copy()
    
    rawdata_test = pq.read_pandas('../input/test.parquet').to_pandas()
    ph1data_test = rawdata_test.iloc[:,ph1_list['signal_id'].values].copy()
    ph2data_test = rawdata_test.iloc[:,ph2_list['signal_id'].values].copy()
    ph3data_test = rawdata_test.iloc[:,ph3_list['signal_id'].values].copy()
    del rawdata_train
    Y1 = ph1_list['target'].copy()
    Y2 = ph2_list['target'].copy()
    Y3 = ph3_list['target'].copy()
    
    del ph1_list, ph2_list, ph3_list, rawmeta_test
    gc.collect()

    return ph1data_test, ph2data_test, ph3data_test, Y1, Y2, Y3

#sigmoid function
def sigmoid(x):
    return 1 / (1 + np.e ** -x)



def initialize_parameters_deep(layers_dims):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    n_x, n_y, n_h, num_h_layers = layers_dims
    
    W1 = 0.01 * np.random.randn(n_h, n_x).astype(np.float32)
    b1 = np.zeros((n_h, 1)).astype(np.float32)
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    
    parameters = {"W1": W1,
                  "b1": b1}
        
    for i in range(2,num_h_layers):
        W = np.random.randn(n_h, n_h).astype(np.float32) * 0.01
        b = np.zeros((n_h, 1)).astype(np.float32)
    
        assert (W.shape == (n_h, n_h))
        assert (b.shape == (n_h, 1))
        
        
        parameters['W'+str(i)] = W
        parameters['b'+str(i)] = b
    
    W_last = np.random.randn(n_y, n_h).astype(np.float32) * 0.01
    b_last = np.zeros((n_y, 1)).astype(np.float32)
    
    assert (W_last.shape == (n_y, n_h))
    assert (b_last.shape == (n_y, 1))

        
    parameters['W'+str(num_h_layers)] = W_last
    parameters['b'+str(num_h_layers)] = b_last
    
    #del W_last, b_last, W, b, W1, b1, i
    #gc.collect()
    
    
    return parameters

def L_model_forward(X, parameters, num_h_layers):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    
    Z1 = np.dot(parameters['W1'],X) + parameters["b1"]
    A1 = np.tanh(Z1)
    #print("W1 shape: ", parameters['W1'].shape)
    #print("b1 shape: ", parameters["b1"].shape)
    #print("A1 shape: ", A1.shape)
    cache = {"Z1": Z1,
             "A1": A1}
    A_prev = A1
    for i in range(2, num_h_layers):
        # Retrieve each parameter from the dictionary "parameters"
        cache['Z'+str(i)] = np.dot(parameters["W"+str(i)],A_prev) + parameters["b"+str(i)]
        cache['A'+str(i)] = np.tanh(cache['Z'+str(i)])
        A_prev = cache['A'+str(i)]
        #print("W"+str(i)+" shape: ", parameters["W"+str(i)].shape)
        #print("b"+str(i)+" shape: ", parameters["b"+str(i)].shape)
        #print("A"+str(i)+" shape: ", cache['A'+str(i)].shape)
        
    cache['Z'+str(num_h_layers)] = np.dot(parameters["W"+str(num_h_layers)],A_prev) + parameters["b"+str(num_h_layers)]
    cache['A'+str(num_h_layers)] = sigmoid(cache['Z'+str(num_h_layers)])
    
    #print("W"+str(num_h_layers)+" shape: ", parameters["W"+str(num_h_layers)].shape)
    #print("b"+str(num_h_layers)+" shape: ", parameters["b"+str(num_h_layers)].shape)
    #print("A"+str(num_h_layers)+" shape: ", cache['A'+str(num_h_layers)].shape)
    
    
    #del A_prev, A1, Z1, i
    #gc.collect()
    
    return cache['A'+str(num_h_layers)], cache



def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    logprobs = (np.multiply(Y,np.log(A2))+(np.multiply((1-Y),(np.log(1-A2)))))
    cost = (-1/m) * np.sum(logprobs)
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    del logprobs, m
    gc.collect()
    
    return cost

def L_model_backward(parameters, cache, X, Y, num_h_layers, dZ_next = [], network_bridge=0):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # Backward propagation: calculate gradients.
    if network_bridge == 1:
        dZ_last = (np.dot(Y.T,dZ_next)) * (1 - np.power(cache["A"+str(num_h_layers)], 2))
    else:
        dZ_last = cache["A"+str(num_h_layers)] - Y
    dW_last = (1/m) * np.dot(dZ_last, cache["A"+str(num_h_layers-1)].T)
    db_last = (1/m) * np.sum(dZ_last, axis=1, keepdims=True)
    dZ_next = dZ_last
    dW_next = dW_last
    db_next = db_last
    grads = {"dW"+str(num_h_layers) : dW_last,
             "db"+str(num_h_layers) : db_last}
    #print("dW"+str(num_h_layers)+" shape: ", grads["dW"+str(num_h_layers)].shape)
    #print("db"+str(num_h_layers)+" shape: ", grads["db"+str(num_h_layers)].shape)
    if num_h_layers > 2:
        for i in range(1, num_h_layers):
            j = num_h_layers - i
            dZ = (np.dot(parameters["W"+str(j+1)].T,dZ_next)) * (1 - np.power(cache["A"+str(j)], 2))
            dW = (1/m) * np.dot(dZ, cache["A"+str(j)].T)
            db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            dZ_next = dZ
            dW_next = dW
            db_next = db
            grads["dW"+str(j)] = dW
            grads["db"+str(j)] = db
            #print("dW"+str(j)+" shape: ", grads["dW"+str(j)].shape)
            #print("db"+str(j)+" shape: ", grads["db"+str(j)].shape)
        
        
    dZ1 = (np.dot(parameters["W"+str(2)].T,dZ_next)) * (1 - np.power(cache["A"+str(1)], 2))
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads["dW1"] = dW1
    grads["db1"] = db1
    
    #print("dW"+str(1)+" shape: ", grads["dW"+str(1)].shape)
    
    del dZ_last, dW_last, db_last, dZ_next, dW_next, db_next, dW1, db1
    gc.collect()
    
    if network_bridge == 2:
        return grads, parameters["W1"], dZ1
    else:
        del dZ1
        gc.collect()
        return grads

def update_parameters(parameters, grads, num_h_layers, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    # Retrieve each parameter from the dictionary "parameters"
    for i in range(1, num_h_layers+1):
    
        # Update rule for each parameter
        parameters["W"+str(i)] = parameters["W"+str(i)] - (learning_rate * grads["dW"+str(i)])
        parameters["b"+str(i)] = parameters["b"+str(i)] - (learning_rate * grads["db"+str(i)])
    
    
    return parameters

def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > .5)
    ### END CODE HERE ###
    
    return predictions

def l_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->TANH]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (number of examples, num_px * num_px * 3)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    costs = []                         # keep track of cost
    
    # Parameters initialization. 
    parameters = initialize_parameters_deep(layers_dims)
    
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> TANH]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters, layers_dims[3])
        
        # Compute cost.
        cost = compute_cost(AL, Y)
    
        # Backward propagation.
        grads = L_model_backward(parameters, caches, X, Y, layers_dims[3])

        # Update parameters.
        parameters = update_parameters(parameters, grads, layers_dims[3], learning_rate)
                
        # Print the cost every 100 training example
        if print_cost and i % 10 == 0:
            print("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 10 == 0:
            costs.append(cost)
    print("Outside of Update for loop")
    print("Cost after iteration %i: %f" %(i, cost))
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

def l_triplelayer_trainmodel(batch_num, layers_dims, layere_dims, learning_rate = 0.005, learning_Erate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->TANH]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (number of examples, num_px * num_px * 3)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    costs = []                         # keep track of cost
    
    # Parameters initialization. 
    parameters1 = initialize_parameters_deep(layers_dims)
    parameters2 = initialize_parameters_deep(layers_dims)
    parameters3 = initialize_parameters_deep(layers_dims)
    
    parametersE = initialize_parameters_deep(layere_dims)
    
    batch_i = 0 #switch out batches of training data
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):
        
        if i % 10 == 0 or i==0:
            if batch_i == batch_num:
                batch_i = 0
            print("Batch Num %i: %i" %(batch_i, batch_i))
            ph1_train, Y = filetondarray("../train_batches/ph1_"+batch_i+".csv", "../train_batches/y1_"+batch_i+".csv")
            ph2_train, Y2 = filetondarray("../train_batches/ph2_"+batch_i+".csv", "../train_batches/y2_"+batch_i+".csv")
            ph3_train, Y3 = filetondarray("../train_batches/ph3_"+batch_i+".csv", "../train_batches/y3_"+batch_i+".csv")
            batch_i += 1

        # Forward propagation for the first 3 input networks: [LINEAR -> TANH]*(L-1) -> LINEAR -> SIGMOID.
        #print("Phase 1 Network")
        AL1, caches1 = L_model_forward(X1, parameters1, layers_dims[3])
        #print("AL1 shape: ", AL1.shape)
        #print("Phase 2 Network")
        AL2, caches2 = L_model_forward(X2, parameters2, layers_dims[3])
        #print("AL2 shape: ", AL2.shape)
        #print("Phase 3 Network")
        AL3, caches3 = L_model_forward(X3, parameters3, layers_dims[3])
        #print("AL3 shape: ", AL3.shape)
        
        """Create intermediate array to feed combining network, 3 networks into one
            N1 ->  |
            N2 ->  | -> AX -> Ne
            N3 ->  |
            """
        
        AX = np.zeros((layere_dims[0], AL1.shape[1]), dtype=np.float32)
        AX[:AL1.shape[0],:] = AL1
        AX[AL1.shape[0]:AL2.shape[0]+AL1.shape[0],:] = AL2
        AX[AL2.shape[0]+AL1.shape[0]:AL3.shape[0]+AL2.shape[0]+AL1.shape[0],:] = AL3
        #print('AX shape: ', AX.shape)
        
        #Forward Propaget combined network segment
        #print("Final Layer Shape")
        ALe, cachesE = L_model_forward(AX, parametersE, layere_dims[3])
        
        # Compute cost.
        cost = compute_cost(ALe, Y)
    
        # Backward propagation.
        #print("Final layer gradients")
        gradsE, Wp, dZ = L_model_backward(parametersE, cachesE, AX, Y, layere_dims[3], network_bridge=2)
        #print("Wp shape: ", Wp.shape)
        #print("dZ shape: ", dZ.shape)
        
        
        #print("Phase 1 gradients")
        grads1 = L_model_backward(parameters1, caches1, X1, Wp[:,:AL1.shape[0]], layers_dims[3], dZ_next=dZ, network_bridge=1)
        #print("Phase 2 gradients")
        grads2 = L_model_backward(parameters2, caches2, X2, Wp[:,AL1.shape[0]:AL2.shape[0]+AL1.shape[0]], layers_dims[3], dZ_next=dZ, network_bridge=1)
        #print("Phase 3 gradients")
        grads3 = L_model_backward(parameters3, caches3, X3, Wp[:,AL2.shape[0]+AL1.shape[0]:AL3.shape[0]+AL2.shape[0]+AL1.shape[0]], layers_dims[3], dZ_next=dZ, network_bridge=1)

        # Update parameters.
        #print("Phase 1 updates")
        parameters1 = update_parameters(parameters1, grads1, layers_dims[3], learning_rate)
        #print("Phase 2 updates")
        parameters2 = update_parameters(parameters2, grads2, layers_dims[3], learning_rate)
        #print("Phase 3 updates")
        parameters3 = update_parameters(parameters3, grads3, layers_dims[3], learning_rate)
        
        #print("Final Layer updates")
        parametersE = update_parameters(parametersE, gradsE, layere_dims[3], learning_Erate)
                
        # Print the cost every 100 training example
        if print_cost and i % 10 == 0:
            print("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 10 == 0:
            costs.append(cost)
    #print("Outside of Update for loop")
    print("Cost after iteration %i: %f" %(i, cost))
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters


In [4]:
#ph1_train, Y1 = filetondarray("../train_batches/ph1_1.csv", "../train_batches/y1_1.csv")


In [5]:
print("ph1 read: ", ph1_train[0:10])
print("Y1 read: ", Y1[0:10])
print(ph1_train.shape)
print(Y1.shape)

NameError: name 'ph1_train' is not defined

In [6]:
layers = [100, 12, 50, 6]
parameters = initialize_parameters_deep(layers)
print(parameters.keys())
print(parameters['W1'].shape)
print(parameters['W2'].shape)
print(parameters['W3'].shape)
print(parameters['W6'].shape)

dict_keys(['W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'W4', 'b4', 'W5', 'b5', 'W6', 'b6'])
(50, 100)
(50, 50)
(50, 50)
(12, 50)


In [7]:
#file_cleanup()

#segment_traindata(2, batchsize_max)

gc.collect()


11

In [8]:
#ph1_train, Y1 = filetondarray("../train_batches/ph1_1.csv", "../train_batches/y1_1.csv")
#ph2_train, Y2 = filetondarray("../train_batches/ph2_1.csv", "../train_batches/y2_1.csv")
#ph3_train, Y3 = filetondarray("../train_batches/ph3_1.csv", "../train_batches/y3_1.csv")

In [9]:
print(ph1_train.shape)
print(Y1.shape)

NameError: name 'ph1_train' is not defined

In [10]:
gc.collect()


119

In [11]:
print(ph1_train.shape)
print(Y1.shape)

NameError: name 'ph1_train' is not defined

In [12]:
#parameters = L_layer_model(ph1_train, Y1, layers_dims, num_iterations = 1000, print_cost = True)

In [13]:
#parameters = L_triplelayer_model(ph1_train, ph2_train, ph3_train, Y1, layers_dims, layere_dims, learning_rate = 0.01, learning_Erate = 0.02, num_iterations = 1000, print_cost=True)